In [14]:
# 라이브러리 import
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import csv
import requests
import pandas as pd
import re
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities #다음화면으로 빨리 넘어갈때
from selenium.common.exceptions import UnexpectedAlertPresentException #selenium 이용시 Alert을 제어
import urllib.request
import random
from selenium.webdriver.chrome.options import Options


In [15]:
#크롤링 시작과 끝 날짜(달 별로 크롤링)
#                                                        각 달의 첫날
first_days=pd.date_range('2022/01/01', '2022/01/31', freq='MS')
#                                                        각 달의 마지막 날
last_days=pd.date_range('2022/10/01', '2022/10/31', freq='M')

#검색 키워드 추가
keyword_list=['치매 실종']

In [16]:
# 크롬드라이버 제어
chrome_options = Options()
chrome_options.add_argument('--no-sandbox') #bypass OS security model
chrome_options.add_argument('--disable-dex-shm-usage')#overcome limited resource problems
# options.add_argument('--headless') # 브라우저 백그라운드 모드


chrome_path = 'chromedriver.exe' #현재 컴퓨터의 크롬드라이버 위치


In [17]:
# 블로그 주소 수집

blog_url_list = []

# 네이버에 키워드 검색
for keyword in keyword_list:
    url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=blog&query={}&oquery={}'.format(keyword, keyword)
    driver = webdriver.Chrome(executable_path = chrome_path, chrome_options=chrome_options)
    driver.implicitly_wait(3)
    
    driver.get(url)
    time.sleep(3)
    
    url_want = 990 #
    
    driver.find_element('xpath','//*[@id="snb"]/div[1]/div/div[2]/a').click() #검색 옵션 클릭
    
    # 날짜 지정
    for k in range(len(first_days)):
        try:
            # 시작과 끝의 년월일 설정
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]/i').click() #직접 입력
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(first_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(first_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(first_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a').click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(last_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button').click()
            
            # 스크롤 조작
            last_height = driver.execute_script('return document.body.scrollHeight')
            
            scroll = (url_want/30)-1 # 한 페이지에 기본으로 30개 글이 나옴
            
            # 스크롤 다운
            for i in range(int(scroll)):
                driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
                # 0부터 연재까지의 스크롤 좌표를 가져와 줘의 의미
                time.sleep(random.uniform(1, 1.7))
                
                new_height = driver.execute_script('return document.body.scrollHeight')
                
                if new_height == last_height:
                    break
                
                last_height = new_height
                
            soup = bs(driver.page_source, 'lxml')
            blog_url = soup.find_all('a', class_='api_txt_lines total_tit')
            
            # url 주소 수집
            for i in blog_url:
                blog_url_list.append(i['href']) # 주소 수집
            
            print('{0} 키워드 {1}년 {2}월'.format(keyword, first_days.year[k], first_days.month[k]), len(blog_url), '개 블로그 url 주소 수집')
            
            driver.execute_script('window.scrollTo(0,0)')
            
        except:
            pass

C:\Users\user\AppData\Local\Temp\ipykernel_14212\2598069413.py:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = chrome_path, chrome_options=chrome_options)


치매 실종 키워드 2022년 1월 990 개 블로그 url 주소 수집


In [18]:
#수집 후 url list 중복 확인 810+990
real_list = set(blog_url_list)
blog_url_list = list(real_list)
len(blog_url_list)

989

In [19]:
#파일 저장(블로그 주소)
k = pd.DataFrame(blog_url_list)
#
k.to_csv('url_list_맛집.csv', encoding='utf-8-sig')

In [20]:
#
url=pd.read_csv('url_list_맛집.csv')
blog_url_list = list(url['0'])

In [21]:
no_naver_blog1=[]

start = time.time()


time_list = [] #시간
review_list = [] #리뷰
comment_list = [] #댓글
like_list = [] #좋아요수
url_list = [] #url주소
count = 0

driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.
for url in blog_url_list: #기본적으로 고치지 않으시지만, n번째 크롤링에서 멈췄다면, blog_url_list를 blog_url_list[n:]으로 고칩니다.
    if 'naver' in url: #네이버 블로그만 수집(다른 사이트가 섞이면 크롤링 에러가 남)
    
        count += 1
        driver.get(url) #url 하나씩 글을 수집할꺼야
        time.sleep(1)

        try:
            # 전체 본문 가지고 와서
            driver.switch_to.frame('mainFrame') #네이버는 mainFrame을 바꿔줘야 수집이 됌
            time.sleep(1)
        
            soup = bs(driver.page_source, 'lxml') #뷰티풀숩을 이용해서 구문 분석(lxml 파싱 이용. 모든 테그에서 데이터를 추출위함)
            postview = soup.find('div', attrs={'id': re.compile('post-view.+')}).get_text()

            reg = re.compile(r'[\s+]') #r은 raw string. \s+는 공백문자
            postview_reg = reg.sub(' ',postview) #공백을 띄어쓰기로 대치해서 본문 글 수집
            try:
                # 좋아요수
                like = driver.find_element("xpath", "//span[@class='u_likeit_list_btn _button btn_sympathy pcol2 off']/em[@class='u_cnt _count']")
                like = like.text
            except:
                like = [] #좋아요 없으면 빈 리스트
            
            try:
                timeline = driver.find_element("xpath", "//span[@class='se_publishDate pcol2']") #수정 안 한 경우
        
            except:
                timeline = driver.find_element("xpath", "//p[@class='date fil5 pcol2 _postAddDate']") #수정 한 경우
            timeline = timeline.text

            try:
                driver.find_element(By.CSS_SELECTOR,'span.btn_arr').click() #댓글 내리는 버튼(태그명.클래스선택자)
                #driver.find_element_by_css_selector('span.btn_arr').click()
                time.sleep(1.7)

                comment_blog = []
                comment = driver.find_element(By.CSS_SELECTOR,'span.u_cbox_contents') #댓글
                #comment = driver.find_elements_by_css_selector('span.u_cbox_contents')
                for i in comment: #댓글 수집
                    com = str(i.text) #
                    com = reg.sub(' ',com) 
                    comment_blog.append(([com]))
            except: 
                comment_blog = []
             
            time_list.append(timeline)
            review_list.append(postview_reg)
            comment_list.append(comment_blog)
            url_list.append(url)
            like_list.append(like)
            time.sleep(random.uniform(1,1.6))
        except UnexpectedAlertPresentException:
            pass
    else:
        no_naver_blog1.append(url)
        print('네이버 블로그 아닌 url: ',url)
        
    if count%10 ==0:
        print('{}개 블로그 크롤링 완료'.format(count))

C:\Users\user\AppData\Local\Temp\ipykernel_14212\3172629000.py:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.


10개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://esmcenter.tistory.com/13419
20개 블로그 크롤링 완료
30개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://slifestory.tistory.com/455
네이버 블로그 아닌 url:  http://blog.yes24.com/blog/blogMain.aspx?blogid=yang412&artSeqNo=16050095
40개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://mangoistree.tistory.com/entry/%EC%A7%80%EB%AC%B8-%EC%82%AC%EC%A0%84%EB%93%B1%EB%A1%9D%EC%A0%9C-%EC%9C%A0%EC%95%84-%EC%95%84%EB%8F%99-%EC%B9%98%EB%A7%A4%ED%99%98%EC%9E%90-%EC%A7%80%EC%A0%81%EC%9E%A5%EC%95%A0%EC%9D%B8%EB%93%B1-%EC%8B%A4%EC%A2%85%EB%8C%80%EB%B9%84-%ED%95%98%EA%B8%B0
50개 블로그 크롤링 완료
60개 블로그 크롤링 완료
70개 블로그 크롤링 완료
80개 블로그 크롤링 완료
90개 블로그 크롤링 완료
100개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://funfunfunny7.tistory.com/396
110개 블로그 크롤링 완료
120개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://guri-guri.tistory.com/495
130개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://platform-1.tistory.com/entry/%EA%B0%80%EC%96%91%EC%97%AD-%EC%8B%A4%EC%A2%85
140개 블로그 크롤링 완료
150개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://esmcenter.tistory.com/13938
150개 블로그

In [22]:
len(time_list)

958

In [23]:
data={'time':time_list, 'review': review_list, 'comment':comment_list, 'like':like_list, 'url':url_list}
df=pd.DataFrame(data)

print('크롤링 소요 시간: ', time.time()-start)
print('총 {}개 블로그 크롤링 완료'.format(count))
df

크롤링 소요 시간:  4054.165376186371
총 958개 블로그 크롤링 완료


,time,review,comment,like,url
0,2022. 9. 12. 10:41,자기경영 뇌세포를 건강하게 노희상 ・ 2022...,[],1,https://blog.naver.com/sang5900/222872526812
1,2022. 3. 1. 12:00,인천경찰청 실종노인.아동찾기 [공지] ★사람을 찾습니다 ...,[],11,https://blog.naver.com/lsykumdo/222660826256
2,2022. 10. 14. 23:28,데이트 일기 소방구조견 승리 대활약 산중 실종 70세 할머니...,[],5,https://blog.naver.com/sjiin77/222900533914
3,2022. 1. 10. 10:00,스마트치료의 이론과 실제 치매체크 한국스마트치료협회...,[],2,https://blog.naver.com/kastedu/222613626787
4,2022. 10. 21. 18:36,건강관리 [수원병점오산동탄 방문요양 방문간호 주간보호 ]치매어...,[],4,https://blog.naver.com/gilcare1/222906613134
...,...,...,...,...,...
953,2022. 8. 2. 1:14,방송리뷰 MBC 4부작 시골스릴러 '멧돼지사냥'1회 ...,[],25,https://blog.naver.com/seyeong0215/222837016900
954,2022. 3. 19. 12:40,CRIME [그것이 알고싶다] 보험설계사 박정자 실종 살인사건...,[],2,https://blog.naver.com/xingfucoco/222676459095
955,2022. 9. 25. 8:21,게시판 충남 서산서 실종 70대 3일만에 숨진채 발견 ...,[],[],https://blog.naver.com/2019december/222883459241
956,2022. 1. 12. 9:23,"제목을 누르면 해당 기사로 이동합니다.대전시, 2022년 지역사회서비스 투자사업 ...",[],8,https://blog.naver.com/bomnal9101/222619483096


In [1]:
Stopwords = ["제공받아", "제공받았", "지원받아", "지원받았", "해당업체", "해당업체",
            "해시태그", "협찬받아", "업체로부터", "체험후기", "무상지급받은", "무상지급"
            ,"카카오톡플러스친구", "톡톡하세요", "네이버톡톡", "무상지원", "카카오톡 플러스친구"
            , "스토리링크", "스토리 링크", "카카오 채널"]

In [ ]:
#Stopwords가 들어간 글은 삭제
#~:review에서 Stopwords가 들어가있으면 제거하라는 의미
data = data[~data['review'].str.contains('|'.join(Stopwords))]
data = data.reset_index(drop=True)
data

In [ ]:
ix = data['review'].str.len()>3000
data=data.loc[~ix]
data=data.reset_index(drop=True)
data

In [ ]:
#기본 전처리
import re
def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)
    return corpus

In [ ]:
#블로그 본문, 댓글 찹쳐 하나의 document로 만들기
data['doc'] = data['review']+' '+data['comment']
data.head()

In [ ]:
data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
data.doc = data.doc.str.replace("\s+", " ") # white space 삭제

data.head()

In [ ]:
data['doc']=data['doc'].str.replace('복사 이웃추가 본문 기타 기능 지도로 보기 전체지도지도닫기 공유하기', '')
data['doc']=data['doc'].str.replace('복사 이웃추가 본문 기타 기능 공유하기', '')

In [ ]:
#csv로 저장
data.to_csv("치매위험_블로그(광고제거).csv", encoding='utf-8-sig')

In [ ]:
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize

In [ ]:
from konlpy.tag import Okt

In [ ]:
#원하는 태그 추출&불용어 제거
okt = Okt()

#불용어
#한국어불용어사전 불러오기
stopwords = pd.read_csv('ko-stopwords.csv')

stopwords=list(stopwords['stopwords'])
stopwords.extend(['에서','고','이다','는','한','씨', "것","거","게","데","이다","건","고","되다","되어다","걸","기",
                  "시","네","듯","랍니","중이","얘","스","도도", "나","수","개","내","기","제","저","인","있다","이렇다",
                  "그렇다","번","위","팅","분","인","링","란","포","두", "진짜", "하다" ,"이다" ,"가다", "이제" ,"들다"
                 ]) #불용어 추가
stopwords=set(stopwords)

#형태소 분석 함수 만들기
def okt_pos_tagging(string):
    pos_words = okt.pos(string, stem=True, norm=True)#형태소 분석, 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag in ['Noun', 'Adjective', 'Verb', 'Adverb'] if word not in stopwords]
    return words


In [ ]:
#형태소 분석 적용
tokenizing_doc=[]
for i in tqdm(data['doc']):
    tokenizing_doc.append(okt_pos_tagging(i))
data['token']=tokenizing_doc#새로운 tokenizing이라는 컬럼에 키워드 형태로 만든 원

In [ ]:
data.token

In [ ]:
#명사 추출 
text_noun=[okt.nouns(doc) for doc in data.doc]
data['text_noun'] = text_noun
data['text_noun'] = data.text_noun.apply(lambda x: [a for a in x if a not in stopwords])

In [ ]:
data.head()

In [ ]:
#리스트 차원 낮춰주는 함수 생성
def flatten(l):
    flatlist = []
    for elem in l:
        if type(elem) == list:
            for e in elem:
                flatlist.append(e)
        else:
            flatlist.append(elem)
    return flatlist

In [ ]:
data1 = data.token.tolist() #series -> list
data2  = flatten(data1)
data2[:20]

In [ ]:
text = nltk.Text(data2)
text

In [ ]:
from wordcloud import WordCloud

In [ ]:
import matplotlib.pyplot as plt
#명사, 동사, 형용사, 부사로 워드클라우드
font_path = '본인 컴퓨터의 폰트 경로'
wordclouds = WordCloud(width=800, height=800, background_color='Black', colormap='Oranges', font_path = 'C:\Windows\Fonts\gulim.ttc')

from collections import Counter
count = Counter(text)
wordclouds = wordclouds.generate_from_frequencies(count)

fig = plt.figure(figsize = (10, 10))
plt.imshow(wordclouds.to_array())
plt.show()

In [24]:
#파일저장
df.to_csv('치매실종.csv',encoding='utf-8-sig')
df

,time,review,comment,like,url
0,2022. 9. 12. 10:41,자기경영 뇌세포를 건강하게 노희상 ・ 2022...,[],1,https://blog.naver.com/sang5900/222872526812
1,2022. 3. 1. 12:00,인천경찰청 실종노인.아동찾기 [공지] ★사람을 찾습니다 ...,[],11,https://blog.naver.com/lsykumdo/222660826256
2,2022. 10. 14. 23:28,데이트 일기 소방구조견 승리 대활약 산중 실종 70세 할머니...,[],5,https://blog.naver.com/sjiin77/222900533914
3,2022. 1. 10. 10:00,스마트치료의 이론과 실제 치매체크 한국스마트치료협회...,[],2,https://blog.naver.com/kastedu/222613626787
4,2022. 10. 21. 18:36,건강관리 [수원병점오산동탄 방문요양 방문간호 주간보호 ]치매어...,[],4,https://blog.naver.com/gilcare1/222906613134
...,...,...,...,...,...
953,2022. 8. 2. 1:14,방송리뷰 MBC 4부작 시골스릴러 '멧돼지사냥'1회 ...,[],25,https://blog.naver.com/seyeong0215/222837016900
954,2022. 3. 19. 12:40,CRIME [그것이 알고싶다] 보험설계사 박정자 실종 살인사건...,[],2,https://blog.naver.com/xingfucoco/222676459095
955,2022. 9. 25. 8:21,게시판 충남 서산서 실종 70대 3일만에 숨진채 발견 ...,[],[],https://blog.naver.com/2019december/222883459241
956,2022. 1. 12. 9:23,"제목을 누르면 해당 기사로 이동합니다.대전시, 2022년 지역사회서비스 투자사업 ...",[],8,https://blog.naver.com/bomnal9101/222619483096


In [35]:
#블로그 주소 수집
blog_url_list=[]
#네이버에 키워드 검색
for keyword in keyword_list:
    url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=blog&query={}&oquery={}'.format(keyword, keyword)
    driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)
    driver.implicitly_wait(3)

    driver.get(url)
    time.sleep(3)

    #한번에 천개가 한계: 네이버 특성
    url_want=990
    
    #검색 옵션 클릭
    driver.find_element('xpath','//*[@id="snb"]/div[1]/div/div[2]/a').click() 

    #날짜 지정
    for k in range(len(first_days)):
        try:
            #시작과 끝의 년월일 설정
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]/i').click() #직접 입력
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(first_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(first_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(first_days.day[k])).click()


            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a').click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(last_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button').click()

            #스크롤 조작
            last_height = driver.execute_script('return document.body.scrollHeight')
            #한페이지에 기본으로 30개 글이 나옴
            scroll = (url_want/30)-1

            #스크롤 다운
            for i in range(int(scroll)):
                driver.execute_script('window.scrollTo(0, document.body.scrollHeight')
                time.sleep(random.uniform(1,1.7))

                new_height=driver.execute_script('return document.body.scrollHeight')

                if new_height == last_height:
                    break
                last_height = new_height

            soup = bs(driver.page_source, 'lxml')
            blog_url = soup.find_all('a', class_='api_txt_lines total_tit')

            #url 주소 수집
            for i in blog_url:
                #주소 수집
                blog_url_list.append(i['href'])

            print('{0} 키워드 {1}년 {2}월'.format(keyword, first_days.year[k], first_days.month[k]), len(blog_url), '개 블로그 url 주소 수집')

            driver.execute_script('window.scrollTo(0,0)')

        except:
            pass



C:\Users\user\AppData\Local\Temp\ipykernel_5768\671444373.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)
